# Graph Neural Networks for Recommendation Systems

In this last TP, you are going to explore the use of Graph Neural Networks (GNNs) for Recommendation Systems.

The dataset you'll use is the dataset MovieLens 100K. This dataset describes people's expressed preference for movies. Each preference is described as a tuple (user, movie, rating). 

Users and movies description are also added. On one side, users are defined by their age, sex, occupation and location. On the other side, movies are described by 19 categories, for which each movie can belong to one or more.

This TP is two-fold. First part concerns the computation and the analysis of the dataset, and will mostly be treated in TD, and second part is about the use of GNNs for movie recommendation.

## Part 1 : Dataset analysis

### Libraries to include

First of all, here are some libraries that you may need during this TP. Some of them are not mandatory, while other ones might not be used at all.

In [ ]:
import math
import copy
import pickle
import glob
import os
import sys
import time
from pathlib import Path
from itertools import zip_longest
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

!pip install gdown
!gdown 1a7ZViLBLoCHKuLP8CSP_5uj0CTABqDaQ

### Setting the random seeds
Used in order to fix the randomness of the experiments. It will be probably useless here, but it is always prefered.

In [ ]:
def set_random_seed(state=1):
    gens = (np.random.seed, torch.manual_seed, torch.cuda.manual_seed)
    for set_state in gens:
        set_state(state)

RANDOM_STATE = 2021
set_random_seed(RANDOM_STATE)

### Read the dataset files using pandas

You start to work here ! First of all, we need to read the data. For this, we are going to use pandas. We also use this stage to remove the unused fields.

The unused fields are : 
- timestamp in ratings
- movieId, movie title, release data, video release date and IMDb URL for movies
- user id and zip code for users

We also use this stage to ont hot encode and normalize our data. Movies categories are already one hot encoded, so we only need to encode the sex and occupation fields. For normalization, we'll normalize the age field.

In [ ]:
def read_data(path):
    files = {}
    path = Path(path)
    for filename in path.glob('*'):
        if filename.suffix == '.data': 
            if filename.stem == 'u':
                columns = ['userId', 'movieId', 'rating', 'timestamp']
                data = pd.read_csv(filename, sep='\t', names=columns, engine='python', encoding='latin-1')
                data.drop(columns = "timestamp", inplace= True)
                files['ratings'] = data
                
        elif filename.suffix == '.item': 
            if filename.stem == 'u':
                #TO DO
                
        elif filename.suffix == '.user': 
            if filename.stem == 'u':
                #TO DO
                
    return files['ratings'], files['movies'], files["users"]

dataset = 'ml-100k'
ratings, movies, users = read_data('data/'+dataset)

We can check what our ratings looks like : 

In [ ]:
ratings.head()

We can do the same for movies and users

In [ ]:
movies.head()

In [ ]:
users.head()

### Analysis

Let's take some time to analyze a bit our dataset. We can for example plot the the histogram of ratings. What do you observe ? 

In [ ]:
ratings["rating"].hist()

__Plot the different features and analyse them.__

In [ ]:
movies.sum().plot.bar()

In [ ]:
#TO DO : plot the histogram for the users sex

In [ ]:
# TO DO : plot the histogram for the users occupation

In [ ]:
# TO DO : plot the histogram for the users age

__Comment the different results you obtain. What kind of biases can you see ? How do you think those biases may perturb your model ? Do you think AI algorithms used also have biases ?__ 


## Part 2 : Preparing the dataset 

No more analysis, now we want to apply machine learning models to recommend movies according to this dataset.

First step is to prepare the dataset in order to make it usable by our models.

The following function take the triples from ratings and convert it into X and y, where X are the pair user/movie, and y is the rating to predict.

__Question : why did we modify the user index ?__


In [ ]:
def create_dataset(ratings, top=None):
    if top is not None:
        ratings.groupby('userId')['rating'].count()

    unique_users = ratings.userId.unique()
    user_to_index = {old: new for new, old in enumerate(unique_users)}
    new_users = ratings.userId.map(user_to_index)

    unique_movies = ratings.movieId.unique()
    movie_to_index = {old: new for new, old in enumerate(unique_movies)}
    new_movies = ratings.movieId.map(movie_to_index)

    n_users = unique_users.shape[0]
    n_movies = unique_movies.shape[0]

    X = pd.DataFrame({'user_id': new_users, 'movie_id': new_movies})
    y = ratings['rating'].astype(np.float32)
    return (n_users, n_movies), (X, y), (user_to_index, movie_to_index)

(n, m), (X, y), _ = create_dataset(ratings)
print(X.head())
print(y.head())
print(f'Embeddings: {n} users, {m} movies')
print(f'Dataset shape: {X.shape}')
print(f'Target shape: {y.shape}')

Now, we have to separate our dataset into 3 parts : train, valid and test. By using the train_test_split() function from sklearn.model_selection, put 60% of the data in training, 20% of the data for validation, and 20% in test.

Hint : the function divide the input into 2 splits...

__Due to the number of entries, you might need to reduce the number of data in training and validation later (personnaly, i have 20% in train and 16% in valid).__

In [ ]:
# TO DO : split the datasets
datasets = {'train': (X_train, y_train), 'val': (X_valid, y_valid), 'test': (X_test, y_test)}
dataset_sizes = {'train': len(X_train), 'val': len(X_valid), 'test': len(X_test)}

print(dataset_sizes)

Now we have our train, valid and test data, we need to compute the nodes of our graph. 

Here, our graph is a bit particular. It is a bipartite graph, where each part represent different entities (users and movies). In order to allow our models to differentiate users and movies, we'll add a new feature, which is going to be 0 if the node is a user, and 1 if the node is a movie. 

We end up by concatenating both matrices into a single one, our node matrix (H in the course)

In [ ]:
def compute_nodes(users, movies):
    #TO DO
    
nodes = compute_nodes(users, movies)

We also compute the corresponding adjacency matrix.

__Question : why do we only use X_train to compute the adjacency matrix ?__

In [ ]:
def compute_adjacency_matrix(X_train):
    #TO DO
    
adjacency_matrix = compute_adjacency_matrix(X_train)
full_adj = compute_adjacency_matrix(X)

__By using matplotlib, plot both adjacency_matrix. What do you observe ?__

Once it is done, we won't use full_adj anymore

In [ ]:
# TO DO : plot the adjacency matrix and the full_adj matrix

## Part 3 : Training section

We now have almost everything we need to train our model. First thing is to convert X_train, y_train, X_valid, y_valid, X_test and y_test into tensors (your job)

In [ ]:
# TO DO : Transform your data into Tensors

Next step is to define a GNN Layer. Complete the following code in order to compute a simple GNN layer \sigma(AHW+b)

In [ ]:
class GNN_layer(nn.Module):
    def __init__(self, input_dim, output_dim=64): 
        super(GNN_layer,self).__init__()
        
        # TO DO :
        # Attributes : input dim and output dim
        
        #Non linearity : ReLU for example
        
        #Layer : Linear layer

        
    def forward(self, nodes, adjacency):
        # TO DO : 
        # AH multiplication
        
        # Apply Linear layer and non linearity

        return out

We now define our model. This model takes as input the nodes, the adjacency matrix, and the list of pair to predict X_(train/valid/test). It returns a value for each pair to predict. 

Complete the model in order to use one gnn layer you previously computed, and one linear layer to predict for each pair. 

In [ ]:
class GNN_model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN_model, self).__init__()
        # TO DO
        # Attributes : intput dim, hidden dim (and output dim ?)
        
        # Add a GNN layer

        
        # Decision layer. Do we need an activation function ? If not, why ? If yes, which one ?

        
    def forward(self, nodes, adjacency, X):
        # TO DO 
        # Apply GNN layer : 
        
        
        # Concatenate each pair of features for X. Given, because we are generous :)
        node_pair = nodes[X].reshape((X.shape[0], 2*self.hidden_dim))
        # Apply decision layer. Not given because we are not THAT generous 
        
        return out

        

Time to train ! Start by transforming the adjacency matrix and the nodes matrix into tensors.

Then, at each eapoch, make one step of training, and one step of validation, and save both losses into lists, in order to plot the training curves.

In [ ]:
nb_epochs = 1000

loss_train = []
loss_valid = []

nodes_tensor = torch.from_numpy(nodes).float().to(device)
adjacency_matrix_tensor = torch.from_numpy(adjacency_matrix).float().to(device)

# TO DO : Define your model, your loss and your optimizer

for i in tqdm(range(nb_epochs)):
    # TO DO 
    # As you've done in previous TP, apply your model on your TRAIN data, compute the loss and backpropagate it. 
    # Also add your loss to your list

    # Now, apply your model on your VALID data. Compute your loss but do NOT backpropagate it. It is only for validation !
    # Also add your lost to your list
    
# TO DO : Apply you model on TEST data and compute the Root Mean-Squared Error (outside the loop)


# TO DO : plot both loss train and loss valid


del model

We now ant to improve our GNN model into a Vanilla GNN \sigma((A+I)HW). No need to change our layer, we only need to change our adjacency matrix ! 

In [ ]:
nb_epochs = 1000

loss_train = []
loss_valid = []

nodes_tensor = torch.from_numpy(nodes).float().to(device)
# TO DO : modify your adjacency matrix tensor. The rest should not change from previous question

# TO DO : Define your model, your loss and your optimizer

for i in tqdm(range(nb_epochs)):
    # TO DO 
    # As you've done in previous TP, apply your model on your TRAIN data, compute the loss and backpropagate it. 
    # Also add your loss to your list

    # Now, apply your model on your VALID data. Compute your loss but do NOT backpropagate it. It is only for validation !
    # Also add your lost to your list
    
# TO DO : Apply you model on TEST data and compute the Root Mean-Squared Error (outside the loop)


# TO DO : plot both loss train and loss valid


del model

Finally, we want to implement the General GNN. This time, we have to modify our GNN layer in order to compute 2 linear layer : one for the adjacency matrix, and one for the identity matrix.

In [ ]:
class GNN_layer_general(nn.Module):
    def __init__(self, input_dim, output_dim=64): 
        super(GNN_layer_general,self).__init__()
        # TO DO :  change the definition of your GNN Layer for General GNN \sigma(AHW1 + IHW2)
        # Attributes : input dim and output dim

        
        #Non linearity : ReLU for example

        
        #Layer : Linear layers

    def forward(self, nodes, adjacency):
        # TO DO : Compute your general GNN

        return out_1+out_2 # <= this is an hint, but you can modify it if this helps you

In [ ]:
class GNN_model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN_model, self).__init__()
        # TO DO : redefine your GNN model. SHould not change from before, but you need to reexecute it
        
        # Attributes : intput dim, hidden dim (and output dim ?)

        # Add a GNN layer

        
        # Out layer. Do we need an activation function ? 
        
        
    def forward(self, nodes, adjacency, X):
        # TO DO :
        # Apply GNN layer : 
        
        
        # Concatenate each pair of features for X. Given, because we are generous :)
        node_pair = nodes[X].reshape((X.shape[0], 2*self.hidden_dim))
        # Apply decision layer. Not given because we are not THAT generous 
        
        return out


In [ ]:
nb_epochs = 1000

loss_train = []
loss_valid = []

nodes_tensor = torch.from_numpy(nodes).float().to(device)
# TO DO : modify your adjacency matrix tensor. The rest should not change from previous question

# TO DO : Define your model, your loss and your optimizer

for i in tqdm(range(nb_epochs)):
    # TO DO 
    # As you've done in previous TP, apply your model on your TRAIN data, compute the loss and backpropagate it. 
    # Also add your loss to your list

    # Now, apply your model on your VALID data. Compute your loss but do NOT backpropagate it. It is only for validation !
    # Also add your lost to your list
    
# TO DO : Apply you model on TEST data and compute the Root Mean-Squared Error (outside the loop)


# TO DO : plot both loss train and loss valid


del model

__The end__
